In [1]:
from pathlib import Path
from src.inspector_git.linker.models import GitProject
from src.inspector_git.linker.transformers import GitProjectTransformer
from src.inspector_git.reader.iglog.readers.ig_log_reader import IGLogReader
from src.run_blame import check_blame
import sys
sys.setrecursionlimit(16336)

def build_and_check_project(iglog_path: str, repo_path: str, compute_annotated_lines :bool) -> GitProject:
    iglog_file = Path(iglog_path)

    with open(iglog_file, "r", encoding="utf-8") as f:
        git_log_dto = IGLogReader().read(f)

    transformer = GitProjectTransformer(
        git_log_dto,
        name=iglog_file.stem,
        compute_annotated_lines=compute_annotated_lines,
    )
    project = transformer.transform()

    if compute_annotated_lines:
        check_blame(project, repo_path)

    return project


In [3]:
project = build_and_check_project(
    "../../test-input/inspector-git/TestInspectorGitRepo.iglog",
    "/home/vortex/Work/BachelorThesis/voyager-target/test_inspector_git_repo/TestInspectorGitRepo",
    True
)


2025-09-17 22:11:36,844 [INFO] src.inspector_git.linker.transformers: Creating GIT project TestInspectorGitRepo
2025-09-17 22:11:36,844 [INFO] src.inspector_git.linker.transformers: Creating commit 1 / 8 (12%)
2025-09-17 22:11:36,845 [INFO] src.inspector_git.linker.transformers: Creating commit 2 / 8 (25%)
2025-09-17 22:11:36,845 [INFO] src.inspector_git.linker.transformers: Creating commit 3 / 8 (37%)
2025-09-17 22:11:36,846 [INFO] src.inspector_git.linker.transformers: Creating commit 4 / 8 (50%)
2025-09-17 22:11:36,846 [INFO] src.inspector_git.linker.transformers: Creating commit 5 / 8 (62%)
2025-09-17 22:11:36,846 [INFO] src.inspector_git.linker.transformers: Creating commit 6 / 8 (75%)
2025-09-17 22:11:36,847 [INFO] src.inspector_git.linker.transformers: Creating commit 7 / 8 (87%)
2025-09-17 22:11:36,847 [INFO] src.inspector_git.linker.transformers: Creating commit 8 / 8 (100%)
2025-09-17 22:11:36,848 [INFO] src.inspector_git.linker.transformers: Done creating GIT project TestIns

In [9]:
import dill
import os

def save_project(project, filename="git_project.dill"):
    filepath = os.path.join("/home/vortex/Work/BachelorThesis/Vortex/src/notebooks", filename)
    with open(filepath, "wb") as f:
        dill.dump(project, f)
    print(f"Project saved to {filepath}")


def load_project(filename="git_project.dill"):
    filepath = os.path.join("/home/vortex/Work/BachelorThesis/Vortex/src/notebooks", filename)
    with open(filepath, "rb") as f:
        project = dill.load(f)
    print(f"Project loaded from {filepath}")
    return project


In [10]:
save_project(project)

RecursionError: maximum recursion depth exceeded

In [6]:
p = load_project()
check_blame(p, "/home/vortex/Work/BachelorThesis/voyager-target/test_inspector_git_repo/TestInspectorGitRepo")

2025-09-17 14:22:54,111 [INFO] src.run_blame: Checking file:(1/4, 25.00%)
2025-09-17 14:22:54,115 [INFO] src.run_blame: Checking file:(2/4, 50.00%)
2025-09-17 14:22:54,120 [INFO] src.run_blame: Checking file:(3/4, 75.00%)
2025-09-17 14:22:54,124 [INFO] src.run_blame: Checking file:(4/4, 100.00%)
2025-09-17 14:22:54,129 [INFO] src.run_blame: Blame check for /home/vortex/Work/BachelorThesis/voyager-target/test_inspector_git_repo/TestInspectorGitRepo complete
2025-09-17 14:22:54,129 [INFO] src.run_blame: Matches: 6 / 6
2025-09-17 14:22:54,130 [INFO] src.run_blame: Match percentage: 100.00%
2025-09-17 14:22:54,130 [INFO] src.run_blame: Files with not found lines: 0
2025-09-17 14:22:54,130 [INFO] src.run_blame: Files with mismatches: 0
2025-09-17 14:22:54,130 [INFO] src.run_blame: Skipped because its binary: 0


Project loaded from /home/vortex/Work/BachelorThesis/Vortex/src/notebooks/git_project.pkl


(set(), set(), set())